In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import timeit
import multiprocessing

In [2]:
popPts = pd.read_csv('pop_wp_Aug_Richard.csv').iloc[:,1:]

In [3]:
popPts.head()

,id,age,sex,hhold,htype,wp,long,lat
0,09001030300i1,9.0,f,09001030300h196,5,09001030300s2,-73.497305,41.057494
1,09001030300i3,8.0,m,09001030300h197,5,09001030300s3,-73.469835,41.064440
2,09001030300i4,17.0,f,09001030300h73,9,09001030100s1,-73.489309,41.054582
3,09001030300i8,8.0,f,09001030300h198,5,09001030300s0,-73.486755,41.045597
4,09001030300i13,14.0,m,09001030300h198,5,09001030100s1,-73.486755,41.045597


In [7]:
#CT Home Point
cthp = pd.read_csv('1_Home_Pts/home_ct_rid.csv').iloc[:,1:]
cthp.head()

,hhold,SrdID
0,09001030300h196,319480
1,09001030300h197,319469
2,09001030300h73,319430
3,09001030300h198,319356
4,09001030300h199,318980


In [4]:
#NJ Home Point
njhp = pd.read_csv('1_Home_Pts/home_nj_rid.csv').iloc[:,1:]
njhp.head()

,hhold,SrdID
0,34017000100h571,345001
1,34017000100h573,344952
2,34017000100h864,344858
3,34017000100h132,345015
4,34017000100h575,344964


In [9]:
njhp.shape

(2231920, 2)

In [9]:
#Ny Home Point
nyhp = pd.read_csv('1_Home_Pts/home_ny_rid.csv').iloc[:,1:]
nyhp.head()

,hhold,SrdID
0,36085001700h148,481995
1,36085001700h149,482024
2,36085001700h152,482031
3,36085001700h150,482015
4,36085001700h154,482033


In [10]:
def match_rdID (hmID, gdf):
    try:
        return (gdf.loc[gdf.hhold == hmID, 'SrdID'].values[0])
    except IndexError:
        pass
      
#def search_gdf (hmID, gdf1, gdf2, gdf3, gdf4, counter):
def search_gdf (hmID, gdf1, counter):
    rdID = 0
    if (rdID == 0):
        rdID = match_rdID(hmID, gdf1)
      #print('insearch ', rdID)
    counter += 1
    return rdID
      
def add_rdIDs(pdf):
    counter = 1
    #start_time = timeit.default_timer()
    #pdf['rdID'] = pdf.apply(lambda row: search_gdf(row['hhold'], hmSplit1, hmSplit2, hmSplit3, hmSplit4, counter), axis=1)
    pdf['RID'] = pdf.apply(lambda row: search_gdf(row['hhold'],nyhp, counter), axis=1)
    #elapsed = timeit.default_timer() - start_time
    #print('job time (seconds): ', elapsed)
    return pdf

def parallelize_pdf(pdf, func):
    num_cores = 7  #leave one free to not freeze machine
    num_partitions = num_cores #number of partitions to split dataframe\
    
    #print('Population Data before split /n', pdf[:3])
    pdf_split = np.array_split(pdf, num_partitions)
    
    pool = multiprocessing.Pool(num_cores)
    pdf = pd.concat(pool.map(func, pdf_split))
    return pdf

    pool.close()
    pool.join()

In [ ]:
#CT Find Road
if __name__=='__main__':
    start_time = timeit.default_timer()
    ct_pop = popPts[popPts.id.str.startswith('09')].copy()
    
    ct_df = parallelize_pdf(ct_pop, add_rdIDs)
    print(ct_df.shape)
    
    elapsed = timeit.default_timer() - start_time
    #print('New Population after pool /n', new_pdf[:3], new_pdf[-3:])
    print('job total time (seconds):', elapsed)
    print('Saving file...')
    ct_df.to_csv('ct_pop_r.csv')

In [17]:
#NJ Find Road
if __name__=='__main__':
    start_time = timeit.default_timer()
    pop = popPts[popPts.id.str.startswith('34')].copy()
    print('Population Size: ', pop.shape)
    ndf = parallelize_pdf(pop, add_rdIDs)
    #Time
    print('After Finnding RoadID',ndf.shape)
    elapsed = timeit.default_timer() - start_time
    #print('New Population after pool /n', new_pdf[:3], new_pdf[-3:])
    print('job total time (seconds):', elapsed)
    print('Saving file...')
    ndf.to_csv('nj_pop_r.csv')

(6148640, 9)
job total time (seconds): 101757.25393314892
Saving file...


In [11]:
#NY Find Road
if __name__=='__main__':
    start_time = timeit.default_timer()
    pop = popPts[popPts.id.str.startswith('36')].copy()
    print('Population Size: ', pop.shape)
    ndf = parallelize_pdf(pop, add_rdIDs)
    print('After Finnding RoadID',ndf.shape)
    #Time
    elapsed = timeit.default_timer() - start_time
    #print('New Population after pool /n', new_pdf[:3], new_pdf[-3:])
    print('job total time (seconds):', elapsed)
    print('Saving file...')
    ndf.to_csv('ny_pop_r.csv')

(13195184, 9)
job total time (seconds): 1172168.438987464
Saving file...


In [12]:
ndf.head()

,id,age,sex,hhold,htype,wp,long,lat,RID
2220343,36085001700i8,2.0,f,36085001700h148,5,36085001100d0,-74.082241,40.635080,481995
2220344,36085001700i9,0.0,f,36085001700h149,5,36085007500d4,-74.081836,40.631879,482024
2220345,36085001700i12,4.0,f,36085001700h152,5,36085003300s1,-74.084482,40.634434,482031
2220346,36085001700i13,9.0,f,36085001700h150,5,36085003300s1,-74.084051,40.633848,482015
2220347,36085001700i15,7.0,f,36085001700h154,5,36085001100s1,-74.082239,40.635773,482033


In [ ]:
#PA Find Road
if __name__=='__main__':
    start_time = timeit.default_timer()
    new_pdf = parallelize_pdf(papop, add_rdIDs)
    print(new_pdf.shape)
    elapsed = timeit.default_timer() - start_time
    print('New Population after pool /n', new_pdf[:3], new_pdf[-3:])
    print('job total time (seconds):', elapsed)
    #print('Saving file...')
    #new_pdf.to_csv('Out_pop_wrid.csv')